<a href="https://colab.research.google.com/github/huntersarnelli/data-exploration/blob/main/Data_deep_dive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I will write some code that gives some ideas for data exploration prior to starting a project. This should allow me to view missing data in detial

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

This will yield a df with columns detialing the missing data and percentage.

In [ ]:
df_train = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')


missing_values = df_train.isnull().sum()
missing_percent = (missing_values/len(df_train))*100

missing_df = pd.DataFrame({
    "Column":missing_values.index,
    "Missing Values": missing_values.values,
    "Missing Value Percent": missing_percent.round(2).values,
    'Data Type': df_train.dtypes.values
})

missing_df = missing_df.sort_values('Missing Values',ascending=False)

print('\nMissing Values summary:')
print(missing_df[missing_df['Missing Values']>=0])

This will yield a tight data plots showing the data distribution via count charts.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20, 15))
plt.subplots_adjust(hspace=0.5, wspace=0.3)

# Create subplots grid (3x3 for 9 columns)
for idx, col in enumerate(df_train.columns, 1):
    plt.subplot(3, 3, idx)

    # Show top 10 categories for columns with many unique values
    if df_train[col].nunique() > 10:
        order = df_train[col].value_counts().index[:10]
        sns.countplot(y=col, data=df_train, order=order)
    else:
        sns.countplot(y=col, data=df_train)

    plt.title(f'{col} Distribution', size=12)
    plt.xlabel('Count')
    plt.ylabel('')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

Now we may want to visualize it using box plots

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Assume your DataFrame is called df and already contains the data

# Define target and feature columns
target = 'Price'
features = [col for col in df_train.columns if col != target]

# Determine grid dimensions for subplots
n_features = len(features)
ncols = 3
nrows = math.ceil(n_features / ncols)

# Create the subplots
fig, axes = plt.subplots(nrows, ncols, figsize=(5 * ncols, 4 * nrows))
axes = axes.flatten()  # flatten in case axes is a 2D array

# Loop through each feature and create the appropriate plot
for i, col in enumerate(features):
    ax = axes[i]

    # Check if column is numeric (int or float) - adjust as needed
    if pd.api.types.is_numeric_dtype(df_train[col]):
        ax.scatter(df_train[col], df_train[target], alpha=0.5)
        ax.set_xlabel(col)
        ax.set_ylabel(target)
        ax.set_title(f'{col} vs {target}')
    else:
        # For categorical data, use a boxplot to show distribution of price
        sns.boxplot(x=df_train[col], y=df_train[target], ax=ax)
        ax.set_xlabel(col)
        ax.set_ylabel(target)
        ax.set_title(f'{col} vs {target}')
        # Rotate x labels for better readability if many categories
        ax.tick_params(axis='x', rotation=45)

# Remove any unused subplots if the total number of features is less than the grid size
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

Now if you want to fill in the missing values, this will fill the object data using mode data. and float data using median.

In [ ]:
for col in df_train.select_dtypes(include=['object']).columns:
    mode_value = df_train[col].mode()[0]
    df_train[col].fillna(mode_value,inplace=True)
    print(f"filled the mssing values in {col} with {mode_value}")

for col in df_train.select_dtypes(include=['float64']).columns:
    mode_value = df_train[col].median()
    df_train[col].fillna(mode_value,inplace=True)
    print(print(f"filled the mssing values in {col} with {mode_value}"))